In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings


from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore  

import pinecone
from pinecone import Pinecone, ServerlessSpec


from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


from dotenv import load_dotenv
import os
from tqdm.autonotebook import tqdm

/Users/saracharmchi/openAI_API/mchatbot/env/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# steps :
""" 
- extract data from pdf : DirectoryLoader
- create text chunks : RecursiveCharacterTextSplitter, split_documents
- download embedding model : hugging face models
- iniitializing vector database : pinecone
- create vector db : pinecone, data, embedding model
- create chain for output :
    + chain : prompt & prompt template
    + llm model : downloaded llama2 quantized model
    + QA technique : use RetrievalQA from langchain :
         use llm as retriever from the similarity search of vector db




"""

' \n- extract data from pdf : DirectoryLoader\n- create text chunks : RecursiveCharacterTextSplitter, split_documents\n- download embedding model : hugging face models\n- iniitializing vector database : pinecone\n- create vector db : pinecone, data, embedding model\n- create chain for output :\n    + chain : prompt & prompt template\n    + llm model : downloaded llama2 quantized model\n    + QA technique : use RetrievalQA from langchain :\n         use llm as retriever from the similarity search of vector db\n\n\n\n\n'

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("/Users/saracharmchi/openAI_API/mchatbot/data")

In [6]:
# create chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500 , chunk_overlap=20 )
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5860

In [8]:
def download_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [9]:
embeddings = download_embedding_model()

/var/folders/bq/4r99blkn4xd8dtgdmlrgk2h00000gn/T/ipykernel_22005/988468174.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
query_test = embeddings.embed_query("hello")
len(query_test)

384

In [11]:
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
#print(PINECONE_API_KEY)

from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_api_key)

In [13]:
index_name = "medical-chatbot"
index = pc.Index(index_name)

In [14]:
index_name

'medical-chatbot'

In [15]:
PineconeVectorStore(index_name=index_name,embedding=embeddings)

vectorstore_from_docs = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=index_name,
    embedding=embeddings
)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11720}},
 'total_vector_count': 11720}

In [16]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11720}},
 'total_vector_count': 11720}

In [17]:
# if you have created db before :
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

In [18]:
#If we already have an index we can load it like this

query = "What is acne?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'page': 39.0, 'source': '/Users/saracharmchi/openAI_API/mchatbot/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(metadata={'page': 39.0, 'source': '/Users/saracharmchi/openAI_API/mchatbot/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(metadata={'page': 38.0, 'source': '/Users/saracharmchi/openAI_API/mchatbot/data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]


In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [22]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x107b1d370>, model='model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/var/folders/bq/4r99blkn4xd8dtgdmlrgk2h00000gn/T/ipykernel_22005/4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query": user_input})


Response :  The answer to the user's question is not provided in the given text.
Response :  Allergies are an overreaction of the immune system to a substance that is typically harmless, such as pollen, dust, or certain foods. When an allergen enters the body, the immune system mistakes it for a harmful invader and releases chemical mediators, including histamine, which cause allergic symptoms. These symptoms can include itching, sneezing, runny nose, congestion, wheezing, coughing, and in severe cases, anaphylaxis, a life-threatening reaction that requires immediate medical attention.
Response :  Acne is a common skin condition characterized by inflammation of oil glands, usually in the face, chest, and back. It can cause redness, swelling, and pimples on the skin. Acne can be caused by hormonal changes, genetics, and environmental factors such as excessive oil production, stress, and humidity. There are several types of acne, including blackheads, whiteheads, papules, pustules, nodul

KeyboardInterrupt: 